# Monitoring Environmental Waste Utilization Scores
# A guide for generating EWU-Dashboards
**Sarah Schmidt & David Laner**  
*January 2023*

## 1. Configuration

### 1.1 Import Packages

In [3]:
import pandas as pd
import numpy as np
import brightway2 as bw
import premise
from mycolorpy import colorlist as mcp
import string

import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

### 1.2 Gather General Information

In [ ]:
# create a list with MS Excel-column names (will be used for preparing calculations in MS Excel)
alphabet = list(string.ascii_uppercase)
excel_cols=[]

for i in range(100):
    n=0
    j=i
    while j-len(alphabet)>=0:
        j=j-len(alphabet)
        n=n+1
    if n>0:
        col=alphabet[n-1]+alphabet[j]
    else:
        col=alphabet[j]
    excel_cols.append(col)

### 1.3 Create Databases